In [1]:
import pandas as pd
from get_specific_data import fetch_data
from datetime import datetime
import yfinance as yf
from scipy.stats import linregress
import numpy as np
import pandasgui
import time
import ta

[I 240830 11:11:15 smartConnect:121] in pool


In [2]:
ticker = "^CRSLDX"

# Fetch the historical data
nifty500_data = yf.download(ticker, start="2023-01-01", end="2024-08-28")
print(nifty500_data.shape)

[*********************100%***********************]  1 of 1 completed


(403, 6)


In [3]:
nifty500_stocks = pd.read_csv('F:\\Bot\\csv_files\\nifty500list.csv')
#filtering csv to EQ and getting only symbol column
nifty500_stocks = nifty500_stocks.loc[nifty500_stocks['Series'] == 'EQ', 'Symbol']

In [4]:
def exponential_regression(x, y):
    #log of closing price
    log_y = np.log(y)

    #get slope, and r_value
    slope, intercept, r_value, _, _ = linregress(x, log_y)

    '''
    r**2 -> coefficient of determinent
    ranges from 0 - 1
    ''' 
    return slope, intercept, r_value**2 

In [5]:
def annualize_return(daily_slope):
    return np.exp(daily_slope * 252) - 1

In [6]:
def calculate_adjusted_return(df, ticker):
    '''
    returns -> list [ticker, slope, annualized_return, adjusted return]
    '''

    #getting last 90 days data
    x = np.arange(90)                   #list from 0-89
    y = df['close'].iloc[-90:].values   #last 90 closing prices


    slope, intercept, r2 = exponential_regression(x, y)
    annualized_return = annualize_return(slope)
    adjusted_return = annualized_return * r2

    return [ticker, slope, annualized_return, adjusted_return]

In [7]:
def check_circuits(df, upper_circuit=20, lower_circuit=20):
    '''
    hit -> if circuit was hit in the last 90 days
    freq -> frequency of circuit hits 
    '''

    hit = False
    freq = 0

    # Filtering the last 90 days data
    days_90 = df.iloc[-90:, :].copy()

    # Calculate the upper and lower circuit levels
    days_90.loc[:, 'upper_circuit'] = days_90['close'].shift(1) * (1 + upper_circuit / 100)
    days_90.loc[:, 'lower_circuit'] = days_90['close'].shift(1) * (1 - lower_circuit / 100)

    # Initialize the circuit columns 
    days_90['Hit Upper Circuit'] = False
    days_90['Hit Lower Circuit'] = False

    # Explicitly cast columns to bool dtype
    days_90['Hit Upper Circuit'] = days_90['Hit Upper Circuit'].astype(bool)
    days_90['Hit Lower Circuit'] = days_90['Hit Lower Circuit'].astype(bool)

   
    days_90.loc[days_90['high'] >= days_90['upper_circuit'], 'Hit Upper Circuit'] = True    
    days_90.loc[days_90['low'] <= days_90['lower_circuit'], 'Hit Lower Circuit'] = True

    # Count the number of hits for both upper and lower circuits
    upper_circuit_hits = days_90['Hit Upper Circuit'].sum()
    lower_circuit_hits = days_90['Hit Lower Circuit'].sum()
    freq = upper_circuit_hits + lower_circuit_hits

    # Determine if there were any hits
    if freq > 0:
        hit = True

    return hit, freq


In [8]:
def get_sma(df, window = 200):

    #SMA calcluation
    df['sma_' + str(window)] = ta.trend.sma_indicator(df['close'], window=window)
    df['sma_' + str(window)+'_above'] = df['close'] > df['sma_' + str(window)] 
    return df

In [ ]:
def calculate_atr(df, window=14):
    atr = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=window)
    # Calculate RMA manually
    rma_atr = atr.average_true_range().ewm(alpha=1/window, min_periods=window, adjust=False).mean()
    return rma_atr

In [9]:
main_df = pd.DataFrame(columns = ['stock_name', 'prev_close', 'slope', 'annualized_return','adjusted_return', 'prev_sma_100_above', 'circuit_hitted', 'circuit_freq', 'prev_atr_20'])

#getting yesterday's date 
yesterday_date = datetime.today().date() - pd.Timedelta(days = 1)


for i in nifty500_stocks:

    try:
        path = 'F:\\Bot\\data dump\\outputs\\'
        data = pd.read_csv(path + i + '.csv') #one day interval data
        data = data.iloc[-400:]

    except Exception as e:
        print(e)
        continue
    
    #Getting sma
    data = get_sma(data, 100)

    #circuit hit
    hit, freq = check_circuits(data)    
    
    #getting atr
    data['atr_20'] = calculate_atr(data, 20)
    print(data.tail())
    #getting slope, annualized return, adjusted_return
    l = calculate_adjusted_return(data, i)


    new_row = {
        'stock_name' : l[0],
        'prev_close' : data['close'].iloc[-1],
        'slope' : l[1],
        'annualized_return' : l[2],
        'adjusted_return' : l[3],
        'prev_sma_100_above' : data['sma_100_above'].iloc[-1],
        'circuit_hitted' : hit, 
        'circuit_freq' : freq, 
        'prev_atr_20' : data['atr_20'].iloc[-1]
    }
    
    
    #adding to dataframe
    main_df.loc[len(main_df.index)] = new_row

360ONE
3MINDIA
ABB
ACC
AIAENG
APLAPOLLO
AUBANK
AARTIIND
AAVAS
ABBOTINDIA
ACE
ADANIENSOL
ADANIENT
ADANIGREEN
ADANIPORTS
ADANIPOWER
ATGL
AWL
ABCAPITAL
ABFRL
AEGISLOG
AETHER
AFFLE
AJANTPHARM
APLLTD
ALKEM
ALKYLAMINE
ALLCARGO
ALOKINDS
ARE&M
AMBER
AMBUJACEM
ANANDRATHI
ANGELONE
ANURAS
APARINDS
APOLLOHOSP
APOLLOTYRE
APTUS
ACI
ASAHIINDIA
ASHOKLEY
ASIANPAINT
ASTERDM
ASTRAZEN
ASTRAL
ATUL
AUROPHARMA
AVANTIFEED
DMART
AXISBANK
BEML
BLS
BSE
BAJAJ-AUTO
BAJFINANCE
BAJAJFINSV
BAJAJHLDNG
BALAMINES
BALKRISIND
BALRAMCHIN
BANDHANBNK
BANKBARODA
BANKINDIA
MAHABANK
BATAINDIA
BAYERCROP
BERGEPAINT
BDL
BEL
BHARATFORG
BHEL
BPCL
BHARTIARTL
BIKAJI
BIOCON
BIRLACORPN
BSOFT
BLUEDART
BLUESTARCO
BBTC
BORORENEW
BOSCHLTD
BRIGADE
BRITANNIA
MAPMYINDIA
CCL
CESC
CGPOWER
CIEINDIA
CRISIL
CSBBANK
CAMPUS
CANFINHOME
CANBK
CAPLIPOINT
CGCL
CARBORUNIV
CASTROLIND
CEATLTD
CELLO
CENTRALBK
CDSL
CENTURYPLY
CENTURYTEX
CERA
CHALET
CHAMBLFERT
CHEMPLASTS
CHENNPETRO
CHOLAHLDNG
CHOLAFIN
CIPLA
CUB
CLEAN
COALINDIA
COCHINSHIP
COFORGE
COLPAL
CAMS
CO

In [22]:
#filtering data
#if sma100 > close
main_df = main_df.loc[main_df['prev_sma_100_above'] == True]

#sorted values
filtered_df = main_df.sort_values(by = 'adjusted_return', ascending = False)[:100]

In [23]:
filtered_df.to_csv(datetime.today().strftime('%Y-%m-%d') + '.csv', index = False)

In [11]:
'''#debug for calclulations

#data from nse file
new_df = pd.read_csv('360One.csv')
new_df = new_df.iloc[::-1].reset_index(drop=True)
new_df.columns = [i.strip().lower() for i in new_df.columns]
new_df.columns
new_df.drop(columns=['series', '52w h', '52w l'], inplace = True)
new_df['date'] = pd.to_datetime(new_df['date'], format='%d-%b-%Y')


#preprocessing
def convert_to_float(val):
    return float(val.replace(',', ''))

# Convert columns to appropriate data types
float_columns = ['open', 'high', 'low', 'prev. close', 'ltp', 'close', 'vwap', 'value']
int_columns = ['volume', 'no of trades']

for col in float_columns:
    new_df[col] = new_df[col].apply(convert_to_float)

for col in int_columns:
    new_df[col] = new_df[col].str.replace(',', '').astype(int)


def calculate_atr(df, window=14):
    atr = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=window)
    # Calculate RMA manually
    rma_atr = atr.average_true_range().ewm(alpha=1/window, min_periods=window, adjust=False).mean()
    return rma_atr




#Getting sma
data = get_sma(new_df, 100)
data['atr_20'] = calculate_atr(data, 20)
#circuit hit
hit, freq = check_circuits(data)


#debuging
print(data.shape)



#getting annualized return
l = calculate_adjusted_return(data, '360One')

print(data.tail())
new_row = {
    'stock_name' : l[0],
    'prev_close' : data['close'].iloc[-1],
    'adjusted_return' : l[1],
    'prev_sma_100_above' : data['sma_100_above'].iloc[-1],
    'circuit_hitted' : hit, 
    'circuit_freq' : freq, 
    'prev_atr_20' : data['atr_20'].iloc[-1]
}'''

"#debug for calclulations\n\n#data from nse file\nnew_df = pd.read_csv('360One.csv')\nnew_df = new_df.iloc[::-1].reset_index(drop=True)\nnew_df.columns = [i.strip().lower() for i in new_df.columns]\nnew_df.columns\nnew_df.drop(columns=['series', '52w h', '52w l'], inplace = True)\nnew_df['date'] = pd.to_datetime(new_df['date'], format='%d-%b-%Y')\n\n\n#preprocessing\ndef convert_to_float(val):\n    return float(val.replace(',', ''))\n\n# Convert columns to appropriate data types\nfloat_columns = ['open', 'high', 'low', 'prev. close', 'ltp', 'close', 'vwap', 'value']\nint_columns = ['volume', 'no of trades']\n\nfor col in float_columns:\n    new_df[col] = new_df[col].apply(convert_to_float)\n\nfor col in int_columns:\n    new_df[col] = new_df[col].str.replace(',', '').astype(int)\n\n\ndef calculate_atr(df, window=14):\n    atr = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=window)\n    # Calculate RMA manually\n    rma_atr = atr.average_true_range().ewm(alph